This is where I am going to write out my code for this project.

Louisville Animal Shelter Data: https://data.louisvilleky.gov/datasets/louisville-metro-ky-animal-service-intake-and-outcome/about
Nationwide Animal Shelter Data: https://www.shelteranimalscount.org/data-request-free-download/

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install pandas matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

df = pd.read_csv('Animal_IO_Data.csv')

print(df.head())

    kennel animalid jurisdiction     intype insubtype               indate  \
0     ID08  A673162        40208      STRAY     FIELD  2021-02-21 00:00:00   
1  FREEZER  A617968        40215  OWNER SUR     FIELD  2022-12-14 00:00:00   
2      N40  A688268        40213  OWNER SUR       OTC  2021-01-19 00:00:00   
3     DW19  A688176        40258     FOSTER    RETURN  2021-01-22 00:00:00   
4   FOSTER  A688325        40218      STRAY       OTC  2021-01-21 00:00:00   

    surreason   outtype  outsubtype              outdate animaltype sex bites  \
0       STRAY       RTO   IN KENNEL  2021-02-21 00:00:00        DOG   N     N   
1       STRAY      EUTH   REQUESTED  2022-12-14 00:00:00        DOG   F     N   
2   OTHER PET  ADOPTION     WALK IN  2021-01-22 00:00:00        CAT   S     N   
3  OWNER DIED  TRANSFER  RESCUE GRP  2021-01-23 00:00:00        DOG   S     N   
4       STRAY    FOSTER         NaN  2021-01-26 00:00:00        DOG   S     N   

  petsize          color            breed so

In [5]:
df2 = pd.read_excel('3 year 21-22-23 SAC Agg.xlsx')
print(df2.head())

  State  Year  Organizations  Intake - Relinquished By Owner Total-Feline  \
0    AK  2021              5                                          886   
1    AK  2022              5                                         1128   
2    AK  2023              5                                          911   
3    AL  2021             17                                         1548   
4    AL  2022             17                                         1419   

   Intake - Stray At Large Total-Feline  Intake - Transferred In Total-Feline  \
0                                  1620                                    85   
1                                  1878                                    51   
2                                  1814                                    45   
3                                  3414                                   707   
4                                  3400                                   816   

   Intake - Owner Intended Euthanasia Total-Feline

In [6]:
df.drop(columns=['kennel', 'animalid', 'jurisdiction', 'sex', 'color', 'breed', 'sourcezipcode', 'ObjectId'], inplace=True)
df = df.dropna()
print (df)

          intype insubtype               indate   surreason   outtype  \
0          STRAY     FIELD  2021-02-21 00:00:00       STRAY       RTO   
1      OWNER SUR     FIELD  2022-12-14 00:00:00       STRAY      EUTH   
2      OWNER SUR       OTC  2021-01-19 00:00:00   OTHER PET  ADOPTION   
3         FOSTER    RETURN  2021-01-22 00:00:00  OWNER DIED  TRANSFER   
5      OWNER SUR  EUTH REQ  2021-09-03 00:00:00       BITES      EUTH   
...          ...       ...                  ...         ...       ...   
52660      STRAY       OTC  2024-08-16 00:00:00       STRAY       RTO   
52661  OWNER SUR       OTC  2024-11-09 00:00:00    EUTH OLD      EUTH   
52664      STRAY     FIELD  2024-12-28 00:00:00       STRAY       TNR   
52665      STRAY       OTC  2024-11-25 00:00:00       STRAY  ADOPTION   
52669      STRAY       OTC  2024-12-05 00:00:00       STRAY  ADOPTION   

       outsubtype              outdate animaltype bites petsize  
0       IN KENNEL  2021-02-21 00:00:00        DOG     N  

In [7]:
df['indate'] = pd.to_datetime(df['indate'])
df_remove =df[~df['indate'].dt.year.isin([2024, 2025])]
df['outdate'] = pd.to_datetime(df['outdate'])
df_remove =df[~df['outdate'].dt.year.isin([2024, 2025])]
print(df)

          intype insubtype     indate   surreason   outtype  outsubtype  \
0          STRAY     FIELD 2021-02-21       STRAY       RTO   IN KENNEL   
1      OWNER SUR     FIELD 2022-12-14       STRAY      EUTH   REQUESTED   
2      OWNER SUR       OTC 2021-01-19   OTHER PET  ADOPTION     WALK IN   
3         FOSTER    RETURN 2021-01-22  OWNER DIED  TRANSFER  RESCUE GRP   
5      OWNER SUR  EUTH REQ 2021-09-03       BITES      EUTH   REQUESTED   
...          ...       ...        ...         ...       ...         ...   
52660      STRAY       OTC 2024-08-16       STRAY       RTO   IN KENNEL   
52661  OWNER SUR       OTC 2024-11-09    EUTH OLD      EUTH   REQUESTED   
52664      STRAY     FIELD 2024-12-28       STRAY       TNR   CARETAKER   
52665      STRAY       OTC 2024-11-25       STRAY  ADOPTION       EVENT   
52669      STRAY       OTC 2024-12-05       STRAY  ADOPTION     WALK IN   

         outdate animaltype bites petsize  
0     2021-02-21        DOG     N   LARGE  
1     2022-

##dff = pd.DataFrame(data)

word_to_drop = "Outcome"
df2 = df2.loc[:, ~df2.columns.str.contains(word_to_drop)]

##print("Updated colums")
##print(df2)


intake_columns = df2.filter(like='Intake' , axis=1)
df2['Total Intake'] = intake_columns.sum(axis=1)
print(df2)

df2_grouping = df2.groupby('State' , as_index=False)['Total Intake'].sum()
print("Grouped by state and sum total:")
print(df2_grouping)

In [8]:
df2.drop(columns=['Organizations', 'Year'], inplace=True)

print(df2)

    State  Intake - Relinquished By Owner Total-Feline  \
0      AK                                          886   
1      AK                                         1128   
2      AK                                          911   
3      AL                                         1548   
4      AL                                         1419   
..    ...                                          ...   
148    WV                                         1062   
149    WV                                         1253   
150    WY                                          975   
151    WY                                         1135   
152    WY                                         1302   

     Intake - Stray At Large Total-Feline  \
0                                    1620   
1                                    1878   
2                                    1814   
3                                    3414   
4                                    3400   
..                               

I am going to combine all the intake columns into one, and do the same for the outtake columns.  I did use AI to help with this.


In [9]:
intake_columns = [col for col in df2.columns if 'Intake' in col]
df2['Combined_National_Intake'] = df2[intake_columns].sum(axis=1)
outtake_columns = [col for col in df2.columns if 'Outcome' in col]
df2['Combined_National_Outcome'] = df2[outtake_columns].sum(axis=1)
print(df2)

    State  Intake - Relinquished By Owner Total-Feline  \
0      AK                                          886   
1      AK                                         1128   
2      AK                                          911   
3      AL                                         1548   
4      AL                                         1419   
..    ...                                          ...   
148    WV                                         1062   
149    WV                                         1253   
150    WY                                          975   
151    WY                                         1135   
152    WY                                         1302   

     Intake - Stray At Large Total-Feline  \
0                                    1620   
1                                    1878   
2                                    1814   
3                                    3414   
4                                    3400   
..                               

In [10]:
df2_grouping = df2.groupby('State' , as_index=False)['Combined_National_Intake'].sum()
df2_grouping2 = df2.groupby('State' , as_index=False)['Combined_National_Outcome'].sum()
df2_grouped = pd.merge(df2_grouping, df2_grouping2, on='State')
print("Grouped by state and sum total:")
print(df2_grouped)

Grouped by state and sum total:
   State  Combined_National_Intake  Combined_National_Outcome
0     AK                     15570                      15566
1     AL                     66126                      68283
2     AR                     36446                      36171
3     AZ                    246363                     253112
4     CA                    950787                     940923
5     CO                    265431                     266580
6     CT                     13538                      12989
7     DE                      5000                       5137
8     FL                    447934                     454275
9     GA                    190386                     185596
10    HI                     62703                      59591
11    IA                     65672                      65714
12    ID                     88248                      90358
13    IL                    129721                     128598
14    IN                    153953    

In [ ]:
combined_df = pd.concat([df, df2], ignore_index=True)
print(combined_df)

          intype insubtype     indate   surreason   outtype  outsubtype  \
0          STRAY     FIELD 2021-02-21       STRAY       RTO   IN KENNEL   
1      OWNER SUR     FIELD 2022-12-14       STRAY      EUTH   REQUESTED   
2      OWNER SUR       OTC 2021-01-19   OTHER PET  ADOPTION     WALK IN   
3         FOSTER    RETURN 2021-01-22  OWNER DIED  TRANSFER  RESCUE GRP   
4      OWNER SUR  EUTH REQ 2021-09-03       BITES      EUTH   REQUESTED   
...          ...       ...        ...         ...       ...         ...   
32840        NaN       NaN        NaT         NaN       NaN         NaN   
32841        NaN       NaN        NaT         NaN       NaN         NaN   
32842        NaN       NaN        NaT         NaN       NaN         NaN   
32843        NaN       NaN        NaT         NaN       NaN         NaN   
32844        NaN       NaN        NaT         NaN       NaN         NaN   

         outdate animaltype bites petsize  ...  \
0     2021-02-21        DOG     N   LARGE  ...   

In [14]:
combined_df_filled = combined_df.fillna("")
print(combined_df_filled)

          intype insubtype     indate   surreason   outtype  outsubtype  \
0          STRAY     FIELD 2021-02-21       STRAY       RTO   IN KENNEL   
1      OWNER SUR     FIELD 2022-12-14       STRAY      EUTH   REQUESTED   
2      OWNER SUR       OTC 2021-01-19   OTHER PET  ADOPTION     WALK IN   
3         FOSTER    RETURN 2021-01-22  OWNER DIED  TRANSFER  RESCUE GRP   
4      OWNER SUR  EUTH REQ 2021-09-03       BITES      EUTH   REQUESTED   
...          ...       ...        ...         ...       ...         ...   
32840                             NaT                                     
32841                             NaT                                     
32842                             NaT                                     
32843                             NaT                                     
32844                             NaT                                     

         outdate animaltype bites petsize  ...  \
0     2021-02-21        DOG     N   LARGE  ...   